# Import Preprocessed Data

In [ ]:
#PROJECT_ROOT = "/content/drive/My Drive/CMPT419/Project" #@param ["/content/drive/My Drive/CMPT419/Project", "/content/drive/My Drive/school-419-project"] {allow-input: true}

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')
#%cd {PROJECT_ROOT}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/CMPT419/Project


In [ ]:
# Import Openface's data
import pandas as pd
import os
savename = "aggregated_openface_data.pickle"

openface_csvs = "./openface_processed/multi"

df = pd.DataFrame()

if os.path.isfile(savename):
  print("Loading combined CSV's from file")
  df = pd.read_pickle(savename)
else:
  for root, dirs, files in os.walk(openface_csvs):
    dirs.clear()  # don't recurse

    for f in [f for f in files if f[-4:] == '.csv']:
      tempdata = pd.read_csv(os.path.join(root, f), skipinitialspace=True)
      tempdata["filename"] = f[0:-4].strip()  # make column with sample number (remove .csv suffix, whitespace)
      df = pd.concat([df, tempdata])

  print("Saving combined CSV's to file")
  df.to_pickle(savename)
df.describe()

Loading combined CSV's from file


,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,eye_lmk_x_0,eye_lmk_x_1,eye_lmk_x_2,eye_lmk_x_3,eye_lmk_x_4,eye_lmk_x_5,eye_lmk_x_6,eye_lmk_x_7,eye_lmk_x_8,eye_lmk_x_9,eye_lmk_x_10,eye_lmk_x_11,eye_lmk_x_12,eye_lmk_x_13,eye_lmk_x_14,eye_lmk_x_15,eye_lmk_x_16,eye_lmk_x_17,eye_lmk_x_18,eye_lmk_x_19,eye_lmk_x_20,eye_lmk_x_21,eye_lmk_x_22,eye_lmk_x_23,eye_lmk_x_24,eye_lmk_x_25,eye_lmk_x_26,...,pose_Ty,pose_Tz,pose_Rx,pose_Ry,pose_Rz,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
count,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,...,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000,22935.000000
mean,155.616133,0.501809,4.605910,0.751813,0.766122,0.036090,0.203211,-0.663631,-0.076915,0.208771,-0.659839,-0.021269,0.232204,511.338561,513.540981,518.662398,523.701761,525.707635,523.873486,518.383976,513.344064,504.688276,507.776730,512.154275,517.574781,523.141496,527.461574,530.542136,527.074441,522.278365,517.093508,512.131908,507.830534,516.282433,518.520946,520.796808,521.776843,520.886959,518.648720,516.373499,...,-85.914567,819.335261,0.223945,0.052617,-0.006233,0.943013,0.363423,0.669315,0.762250,0.390661,0.458547,0.256579,0.720977,0.569083,0.606051,0.497036,0.386099,0.456355,0.192225,0.584426,0.434419,0.331175,0.738609,0.649706,0.488816,0.367342,0.284500,0.300196,0.183824,0.418400,0.294790,0.399913,0.581600,0.286767,0.381905,0.208197,0.404229,0.323261,0.406584,0.318552
std,164.296348,0.710695,4.882572,0.367902,0.423305,0.332307,0.171565,0.378508,0.330183,0.175701,0.375970,0.370160,0.196829,314.182935,314.628445,315.575512,316.489397,316.813842,316.420605,315.363367,314.479068,313.387881,313.767768,314.390277,315.290230,316.333173,317.194763,317.841956,317.070529,316.074031,315.100014,314.298609,313.714367,315.077610,315.475866,315.906816,316.114550,315.972919,315.568426,315.143290,...,125.684591,524.629290,0.254732,0.481073,0.240291,0.899553,0.685331,0.760899,0.892518,0.580531,0.766686,0.447434,0.774765,0.727288,0.822098,0.669445,0.635610,0.659855,0.426059,0.781036,0.682406,0.545373,0.439402,0.477072,0.499886,0.482091,0.451186,0.458353,0.387349,0.493307,0.455958,0.489891,0.493307,0.452262,0.485864,0.406027,0.490753,0.467732,0.491207,0.465925
min,1.000000,0.000000,0.000000,0.000000,0.000000,-0.805486,-0.448546,-0.999994,-0.914196,-0.414886,-0.999980,-0.988000,-0.473000,16.700000,18.300000,22.300000,26.400000,28.200000,27.100000,22.600000,18.400000,12.700000,13.900000,16.400000,20.400000,25.400000,29.600000,30.600000,29.500000,25.000000,20.700000,17.100000,13.900000,21.200000,23.000000,24.900000,25.600000,24.800000,22.900000,21.100000,...,-934.700000,187.400000,-2.132000,-1.505000,-2.172000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [ ]:
# filter out datas with success = 1 or confidence < 0.95
df = df[(df['confidence'] >=0.65) & (df['success'] == 1)]
df.reset_index(drop=True, inplace=True)
# print(df.filename.unique())
# print(len(df.keys()))

In [ ]:
# Import audio data
audio_df = pd.read_csv("audio-features.csv").drop("Unnamed: 0", axis=1)
audio_df.head()

,mean_pitch,min_pitch,max_pitch,mean_intensity,root_mean_square,center_of_gravity,kurtosis,skewness,standard_deviation,mean_f1_frequency,mean_f2_frequency,mean_speech_rate,number_of_silent_intervals,filename,estimated_frame_number
0,290.447878,223.029898,420.621402,83.265478,0.291275,629.042192,60.851689,4.878547,475.672620,605.231710,1634.643303,0.49362,2,nonflirt-PJP-C7TCaM8-6,0
1,288.354722,223.129737,420.629276,83.295576,0.292287,630.297852,60.325458,4.840207,476.914570,626.793360,1624.179187,0.49362,2,nonflirt-PJP-C7TCaM8-6,1
2,286.543449,223.243593,420.609286,83.482699,0.298652,662.504028,68.563877,4.773974,496.830745,636.716477,1611.940717,0.49362,2,nonflirt-PJP-C7TCaM8-6,2
3,284.431711,223.167087,420.564528,83.700942,0.306251,683.002668,58.698875,4.373654,499.577060,648.038718,1610.828022,0.49362,2,nonflirt-PJP-C7TCaM8-6,3
4,282.287917,223.092497,420.501990,83.765693,0.308542,681.849847,49.727052,4.123383,495.618783,657.417870,1599.743717,0.49362,2,nonflirt-PJP-C7TCaM8-6,4


In [ ]:
# Check datas consistency
clips_name = df.filename.unique()
audios_name = audio_df.filename.unique()
df = df[df['filename'] != 'nonflirt-ImEnWAVRLU0-2']
df = df[df['filename'] != 'nonflirt-v2VgA_MCNDg-4']

# if len(clips_name) != len(audios_name):
#   missing_video = set(audios_name)-set(clips_name)
#   print(missing_video)
missing_audio = set(df.filename.unique())-set(audio_df.filename.unique())
print(missing_audio)
  # df = df[df['filename'] != missing_video.pop()]
  # audio_df = audio_df[audio_df['filename'] != missing_audio]

print(len(df.filename.unique()))
print(len(audio_df.filename.unique()))


set()
103
103


# Feature Selection

In [ ]:
print(df.keys())
# feature_columns = []
# features = train.drop('filename',axis=1).keys()

Index(['frame', 'face_id', 'timestamp', 'confidence', 'success', 'gaze_0_x',
       'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y',
       ...
       'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', 'AU28_c',
       'AU45_c', 'filename', 'Class'],
      dtype='object', length=336)


# Compute DTW

In [ ]:
# required packages
!pip install fastdtw
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from tqdm import tqdm, trange

In [ ]:
clips = pd.concat([df.loc[:,"filename"], df.loc[:,"frame":"gaze_angle_y"], df.loc[:,"pose_Tx":"pose_Rz"], df.loc[:,"AU01_r":"AU45_c"]],
                  axis=1)

In [ ]:
def independent_dtw(clip1, clip2, features):
  overall_score = 0
  clip1 = clip1[~clip1.isin([np.nan, np.inf, -np.inf]).any(1)]
  clip2 = clip2[~clip2.isin([np.nan, np.inf, -np.inf]).any(1)]
  for feature in features:
    if feature != 'filename' and feature != 'frame':
      # print(clip1[feature], clip2[feature])
      distance, path = fastdtw(clip1[feature], clip2[feature], dist=euclidean)
      # print(path)
      overall_score+=distance
  return overall_score

# test
clip1 = clips[clips['filename'] == 'nonflirt-PJP-C7TCaM8-6']
clip2 = clips[clips['filename'] == 'flirt-qkq9k5FewvI-1']
test_dtw1 = independent_dtw(clip1, clip2, clips.columns)
clip1 = audio_df[audio_df['filename'] == 'nonflirt-PJP-C7TCaM8-6']
clip2 = audio_df[audio_df['filename'] == 'flirt-qkq9k5FewvI-1']
test_dtw2 = independent_dtw(clip1, clip2, audio_df.columns)

print(test_dtw1+test_dtw2)

273785.2142364928


In [ ]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

x = np.array([[1,1], [2,2], [3,3], [4,4], [5,5]])
y = np.array([[2,2], [3,3], [4,4]])
distance, path = fastdtw(x, y, dist=euclidean)
print(distance)


2.8284271247461903


In [ ]:
# This would be the final matrix that gives how similar between
# each two clips
def produce_Similarity_Matrix(df, audio_df): # for multimodal
  # This data is missing from OpenFace
  clips_name = df.filename.unique()
  audios_name = audio_df.filename.unique()
  length = len(clips_name)
  features_face = clips.columns
  features_audio = audio_df.columns


  Similarity_Matrix = np.zeros((length,length))
  for col in trange(length, leave=False, position=0):
    for row in trange(length, leave=False, position=0):
      # create a Similarity_Matrix, ie         clip1 clip2 clip3
      #                                 clip1    0    ...   ...
      #                                 clip2   ...    0    ...
      #                                 clip3   ...   ...    0
      clip_name1 = clips_name[col]
      clip1 = clips[clips['filename'] == clip_name1]
      clip_name2 = clips_name[row]
      clip2 = clips[clips['filename'] == clip_name2]
      overall_score1 = independent_dtw(clip1, clip2, features_face)

      clip1 = audio_df[audio_df['filename'] == clip_name1]
      clip2 = audio_df[audio_df['filename'] == clip_name2]
      overall_score2 = independent_dtw(clip1, clip2, features_audio)

      Similarity_Matrix[col][row] = overall_score1+overall_score2
  return Similarity_Matrix

In [ ]:
Similarity_Matrix = produce_Similarity_Matrix(df, audio_df)
sm = Similarity_Matrix
print(sm)

[[      0.           39700.39783521   14579.874227   ...  833440.97968611
   871546.94994018 1512008.68622942]
 [  39706.48120821       0.           12773.026797   ...  798973.20611083
   761487.00584189 1205306.81926377]
 [  14579.808186     12777.355809         0.         ...  265555.152906
   128651.674925    274633.957958  ]
 ...
 [ 833438.31062211  811749.55311101  265592.997991   ...       0.
   559899.04911218  931043.799445  ]
 [ 871575.08818518  791197.36868151  128638.481901   ...  564785.91985668
        0.          843583.3707392 ]
 [1512010.48727242 1271360.20612628  274632.648078   ...  932919.05460808
   845676.94519348       0.        ]]


In [ ]:
# Save the result
import pickle
with open('Similarity_Matrix_total', 'wb') as f:
    pickle.dump(Similarity_Matrix, f)

In [ ]:
# Load the result
import pickle
with open("Similarity_Matrix_total", "rb") as f:
  Similarity_Matrix = pickle.load(f)

FileNotFoundError: ignored

# K-Means Clustering

In [ ]:
# import packages
# !pip install tslearn
from tslearn.clustering import TimeSeriesKMeans
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## look for optimal k to see how many types of flirtatiou/non-flirtatiou is detected

In [ ]:
# We can see that the elbow is around 3 which is the optimal number of clusters
np.random.seed(1000)
Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = TimeSeriesKMeans(n_clusters=k, metric="dtw", max_iter=5)
    km = km.fit(Similarity_Matrix)
    Sum_of_squared_distances.append(km.inertia_)

plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

# 
km = TimeSeriesKMeans(n_clusters=3, metric="dtw")
km.fit(Similarity_Matrix)

labels = km.labels_
print(labels)

KeyboardInterrupt: ignored

## Visulization

In [ ]:
print(clips_name)

## might not use

In [ ]:
dropIndices = ['filename', 'frame']
Train = df.drop(dropIndices,axis=1)
Train = Train.to_numpy()
# We want to visualize if there is a clear distinguish between flirting actions or not
kmeans = TimeSeriesKMeans(n_clusters=2, random_state=1)
clusters = kmeans.fit_predict(Train)
num_clusters = kmeans.n_clusters
cluster_centers = kmeans.cluster_centers_
# print(np.sum(clusters == 0))

# for cluster_num in range(num_clusters):
#     sns.scatterplot(
#         x=Train[clusters == cluster_num, 0], y=Train[clusters == cluster_num, 1]
#     )

# plt.scatter(
#     x=cluster_centers[:, 0], y=cluster_centers[:, 1],
#     s=250, marker='x', c='blue'
# )
# plt.legend(['cluster 1', 'cluster 2', 'centroid'])
# plt.show()

### Social Signals

In [ ]:
# # Find the frame that belongs to the center of the 
# closest_points, _ = pairwise_distances_argmin_min(cluster_centers.reshape(2,333), Train)
# print('The closest points to each centroids are', closest_points)

# closest_points_df = pd.DataFrame().reindex_like(df).iloc[0:0]
# # print(df.index)
# for closest_point in closest_points:
#     gif = df.loc[[closest_point]]
#     closest_points_df = closest_points_df.append(gif)

# closest_points_df

Comparing this to our annotation we found:
- 

# K-Means Clustering

In [ ]:
!pip install tslearn
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
from collections import defaultdict
from tqdm import tqdm
import pickle

ts_seed = 85510309
ss_seed = 10474245


start, stop = 5, 15

savename = f"kmeans_and_silhouette_score_from_{start}_to_{stop}.pickle"

score_per_run = defaultdict(list)

if os.path.isfile(savename):
  print(f"Loading saved k-means dict and silhouette scores from {save_location}")
  with open(savename, 'rb') as f:
    score_per_run = pickle.load(f)
else:
  for i in tqdm(range(start, stop)):
    km = TimeSeriesKMeans(n_clusters=i, metric="dtw", random_state=ts_seed)
    km.fit(Similarity_Matrix)
    ss = silhouette_score(Similarity_Matrix, labels=km.labels_, metric='dtw', random_state=ss_seed)

    score_per_run[i] = [ss, km]

  print(f"Saving {savename}")
  with open(savename, 'wb') as f:
    pickle.dump(score_per_run, f)

     |████████████████████████████████| 778kB 5.2MB/s 


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)

100%|██████████| 10/10 [01:32<00:00,  9.22s/it]

Saving kmeans_and_silhouette_score_from_5_to_15.pickle


In [ ]:
# print(sorted([3,1,7]))  # sort is ascending
s = sorted(score_per_run, key=lambda k: score_per_run[k][0])  # sort by silhouette score
print(s)
print(f"Best (smallest) silhouette score is with {s[0]} categories")

num_metrics = s[0]

[8, 6, 14, 9, 10, 12, 11, 13, 7, 5]
Best (smallest) silhouette score is with 8 categories


# K-Nearest Neighbours Classification

In [ ]:
# import packages
# https://www.saedsayad.com/k_nearest_neighbors.htm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
def is_flirting(filename):
  return filename.startswith("flirt-")

# label all clips, flirting: 1, non-flirting: 0
# df['Class'] = 0
# df.loc[is_flirting(df['filename'].str), 'Class'] = 1
# print(df['Class'].value_counts())
num_clips = df.filename.unique()

labels = []
for i in range(0, len(num_clips)):
  clip_filename = num_clips[i]
  if is_flirting(clip_filename):
    label = 1
  else:
    label = 0
  labels.append(label)

features = df.keys()
# print(features)
# Split the dataset into train (80%) and test (20%) subsets.
X_train, X_test, y_train, y_test = train_test_split(
    Similarity_Matrix, labels, test_size=0.2, shuffle=False
)

# Create a classifier with k=1
knn = KNeighborsClassifier(n_neighbors=10)

# Train
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('k-NN Accuracy (without Cross-Validation):', knn.score(X_test, y_test))

k-NN Accuracy (without Cross-Validation): 0.6666666666666666


## Dimension Deduction and Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import (KNeighborsClassifier,
                               NeighborhoodComponentsAnalysis)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# Reference:
# https://scikit-learn.org/stable/auto_examples/neighbors/plot_nca_dim_reduction.html

np.random.seed(1000)
n_neighbors = 24
random_state = 0

# Split into train/test
X_train, X_test, y_train, y_test = \
    train_test_split(Similarity_Matrix, labels, test_size=0.2, stratify=labels,
                     random_state=random_state)

dim = len(Similarity_Matrix[0])
n_classes = 2

# Reduce dimension to 2 with NeighborhoodComponentAnalysis
nca = make_pipeline(StandardScaler(),
                    NeighborhoodComponentsAnalysis(n_components=2,
                                                   random_state=random_state))

# Use a nearest neighbor classifier to evaluate the methods
knn = KNeighborsClassifier(n_neighbors=n_neighbors)


name = 'NCA'
model = nca
plt.figure()

# Fit the method's model
model.fit(X_train, y_train)

# Fit a nearest neighbor classifier on the embedded training set
knn.fit(model.transform(X_train), y_train)
y_pred = knn.predict(model.transform(X_test))
# Compute the nearest neighbor accuracy on the embedded test set
acc_knn = knn.score(model.transform(X_test), y_test)

# Embed the data set in 2 dimensions using the fitted model
X_embedded = model.transform(Similarity_Matrix)
# Plot the projected points and show the evaluation score
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=labels, s=30)
plt.title("{}, KNN (k={})\nTest accuracy = {:.2f}".format(name,
                                                          n_neighbors,
                                                          acc_knn))
plt.show()

NameError: ignored

In [ ]:
for i in range(0, len(y_pred)):
  offset = len(X_train)
  print(y_pred[i], num_clips[offset+i], y_test[i])

1 flirt-qkq9k5FewvI-4 1
0 nonflirt-pNqEDwpIm9-1 0
1 flirt-1 1
1 flirt-True.Blood.S01E02.720p.HDTV.x264-2HD-1 1
0 nonflirt-lyvAjZw6O_Q-10 0
0 nonflirt-PJP-C7TCaM8-6 0
0 nonflirt-v2VgA_MCNDg-0 0
0 flirt-ImEnWAVRLU0-0 1
1 flirt-1v98t9YKPmc-1 1
1 nonflirt-PJP-C7TCaM8-0 0
0 flirt-WAkZz1hV7_c-3 1
1 nonflirt-PJP-C7TCaM8-1 0
1 flirt-5Vl8Wp3aBwQ-1 1
1 nonflirt-2 0
1 nonflirt-Dyri6DRZltk 0
1 flirt-1v98t9YKPmc-2 1
1 flirt-4 1
1 nonflirt-1 0
1 flirt-qUxXbU2ModY-0 1
1 flirt-2 1
1 flirt-True.Blood.S01E01.720p.HDTV.x264-2HD-2 1


Analyzation (note that this is compared to the Clips&Annotation.ipynb): 
1. PJP-C7TCaM8-0  
This is pretty surprising because the boy detected looks totally angry instead of having any intention of doing flirtious action.

2. WAkZz1hV7_c-3  
This is another unexpected results since his head turns right + looks away then back + lip droo + head small movement with smile and also smacking lips. The combinations of these features should give a positive result, label it as a flirting action

3. Dyri6DRZltk  
This one might due to the hand movement and the high picth voice

4. ImEnWAVRLU0-0  
I think that this was predicted as non-flirting because there is no wink or the clip was actually supposed to be labelled as non-flirting.

5. nonflirt-2  
Somewhat surprising because the man with the cellphone's tone of voice is  is agitated. Maybe the model detected the presence of two people in the frame.

6. nonflirt-1  
Somewhat surprising because the child's facial expression seems angry.


# Evaluation

In [ ]:
# import packages
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Train model with a cross-validation (CV) generator with 5 folds. Note: the 
# CV generator uses `sklearn.model_selection.StratifiedKFold`.
# cv_scores = cross_val_score(knn, Similarity_Matrix, labels, cv=5)
cv_scores = cross_val_score(knn, X_embedded, labels, cv=5)

print('Cross-Validation Scores:', cv_scores)
print('Mean Cross-Validation Score:{}'.format(np.mean(cv_scores)))

Cross-Validation Scores: [0.61904762 0.76190476 0.76190476 0.7        0.7       ]
Mean Cross-Validation Score:0.7085714285714285


## Confusion Matrix & Classification Report

In [ ]:
# Reference:
# https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[9 1]
 [3 8]]
              precision    recall  f1-score   support

           0       0.75      0.90      0.82        10
           1       0.89      0.73      0.80        11

    accuracy                           0.81        21
   macro avg       0.82      0.81      0.81        21
weighted avg       0.82      0.81      0.81        21



The results show that our KNN algorithm was able to classify all the 21 records in the test set with 81% accuracy, which is very good. Although the algorithm performed very well with this dataset, don't expect the same results with all applications. As noted earlier, KNN doesn't always perform as well with high-dimensionality or categorical features.

## Comparing Error Rate with the K value